In [ ]:
from importlib import reload
from helper import run
import ecc
import helper

from ecc import FieldElement, Point

In [ ]:
a = FieldElement(num=0, prime=223)
b = FieldElement(num=7, prime=223)
x = FieldElement(num=192, prime=223)
y = FieldElement(num=105, prime=223)
p1 = Point(x, y, a, b)
print(p1)

In [ ]:
reload(ecc)
run(ecc.ECCTest('test_on_curve'))